# A quick example of standard detection pipeline

 
# Gravitational Wave Matched Filter

The Gravitational Wave (GW) matched filter is a crucial tool in the detection of gravitational waves.  The matched filter operates by correlating the incoming data stream from the gravitational wave detector with a theoretically predicted waveform template. This template is derived from the expected signal characteristics based on gravitational wave theory and the properties of potential astrophysical sources.

## Implementation

1. **Template Generation**: Construct theoretical waveform templates corresponding to different gravitational wave sources using numerical relativity simulations or analytical approximations.

2. **Cross-correlation**: Slide the generated templates over the detector data stream and compute the cross-correlation between the templates and the data at each point in time.

3. **Signal Identification**: Look for significant peaks in the correlation output that exceed a predefined threshold. These peaks indicate the presence of a potential gravitational wave signal.

4. **Parameter Estimation**: Once a candidate event is identified, refine the estimation of its parameters (such as source location, masses, and orbits) using additional statistical techniques.

## Advantages

- **Optimal SNR Enhancement**: The matched filter provides the optimal signal enhancement by maximizing the correlation between the expected signal and the detector output.
  
- **Statistical Significance**: By setting appropriate threshold levels, the matched filter helps in distinguishing true gravitational wave events from random fluctuations in the detector noise.

- **Robustness**: It can handle various sources of noise and signal distortions, making it a robust technique for gravitational wave detection.

 

The objective of this exercise is to gain a practical understanding of the Gravitational Wave (GW) matched filter technique and its application in the detection of gravitational wave signals.
We will use materials from the GWOSC workshop:
    https://github.com/gw-odw/odw-2022/tree/main

In [1]:
import sys
sys.path = ['/cvmfs/software.igwn.org/conda/envs/igwn-py39/lib/python3.9/site-packages'] + sys.path
import gwpy
from gwpy.timeseries import TimeSeries
from matplotlib import pyplot as plt
%matplotlib inline
import numpy
import pylab
import pandas as pd
import seaborn as sns


/cvmfs/software.igwn.org/conda/envs/igwn-py39/lib/python3.9/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS
/cvmfs/software.igwn.org/conda/envs/igwn-py39/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/cvmfs/software.igw

In [2]:
https://github.com/gw-odw/odw-2022/tree/main

SyntaxError: invalid syntax (3846423250.py, line 1)

In [ ]:
MDC_PATH = "/cvmfs/et-gw.osgstorage.org/et-gw/PUBLIC/MDC1/data"
DATASETS = ['E0','E1','E2','E3','C1','C2']
CHANNELS = {n : f'{n}:STRAIN' for n in DATASETS}

In [ ]:

e0 = TimeSeries.read(MDC_PATH+'/E1/E-E1_STRAIN_DATA-1000000000-2048.gwf', 'E1:STRAIN')
# The TimeSeries object contains various useful metadata
print(e0[1:10])

In [ ]:
signals = pd.read_csv('../metadata/list_etmdc1_snr.txt', sep=' ')

In [ ]:
# Plot distribution of SNRs
print(f'There are {sum(signals["SNR"]>15)} signals with SNR>15.')
fig = plt.figure(figsize=(6,4))
ax = fig.gca()
sns.histplot(signals['SNR'], log_scale=(True,False), kde=True, ax=ax)
plt.show()


In [ ]:
! pip install -q 'pycbc' --user

In [ ]:
from pycbc.waveform import get_td_waveform

# the "approximant" (jargon for parameterized waveform family).
# IMRPhenomD(a phenomenological Inspiral–Merger–Ringdown wafeform model) is defined in the frequency domain, but we'll get it in the time domain (td).
# It runs fast, but it doesn't include effects such as non-aligned component spin, or higher order modes.
apx = 'IMRPhenomD'

In [ ]:
sample_rate=8192
 
hp1, _ = get_td_waveform(approximant=apx,
                         mass1=10,
                         mass2=10,
                         delta_t=1.0/sample_rate,
                         f_lower=25)

In [ ]:
hp1 = hp1 / max(numpy.correlate(hp1, hp1, mode='full'))**0.5

In [ ]:
import lal
def read_without_warnings(cachefile, channel, **kwargs):
    """Reads cache file while suppressing spurious LAL warnings.
    Wraps gwpy.timeseries.TimeSeries.read

    Args:
        cachefile (str): path to cache file
        channel (str): Channel name
        **kwargs: additional arguments passed to TimeSeries.read
    """
    prev_level = lal.GetDebugLevel()
    lal.ClobberDebugLevel(0)
    result = TimeSeries.read(cachefile, channel, **kwargs)
    lal.ClobberDebugLevel(prev_level)
    return result

def get_strain(tc, t_before = 4, t_after = 1):
    start_time = tc - t_before
    end_time = tc + t_after
    h = read_without_warnings('../caches/E1.lcf','E1:STRAIN',start=start_time, end=end_time)
    return h
 

In [ ]:
tc=1000604258.71989
h=get_strain(tc, t_before = 16, t_after = 16)

In [ ]:
pylab.figure()
pylab.title("The strain h")
pylab.plot( h)
pylab.xlabel('Time (s)')
pylab.ylabel('Normalized amplitude')
print (len(h))

In [ ]:
%matplotlib inline
# note that in this figure, the waveform amplitude is of order 1.
# The duration (for frequency above f_lower=25 Hz) is only 3 or 4 seconds long.
# The waveform is "tapered": slowly ramped up from zero to full strength, over the first second or so.
# It is zero-padded at earlier times.
pylab.figure()
pylab.title("The waveform hp1")
pylab.plot(hp1.sample_times, hp1)
pylab.xlabel('Time (s)')
pylab.ylabel('Normalized amplitude')



In [ ]:
# Shfrom pycbc.waveform import get_fd_waveform
from pycbc.waveform import get_fd_waveform
from pycbc.filter import matched_filter
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation

# Calculate the component mass of each black hole in the detector frame
cmass = 10
# This is a frequency domain waveform generator. It has a very similar syntax to the time domain
# waveform function used in prior tutorials. This function returns both a plus and a cross
# polarization waveform, but we will just use the plus polarization in building our template
# as these are only different by a phase offset in this specific case.
hp, _ = get_fd_waveform(approximant="IMRPhenomD",
                         mass1=cmass, mass2=cmass,
                         f_lower=20.0, delta_f=5)
hp.resize(len(psd[ifo]))

psd=
 # For each observatory use this template to calculate the SNR time series
matched_filter(hp, h, psd, low_frequency_cutoff=20)